# 🚂 Sistema Ollama + FastAPI
## Modelos de IA para Análise de Transporte Ferroviário

Este notebook permite usar modelos de linguagem local para análises de transporte:
- **TinyLLama**: Modelo rápido e leve (1.1B parâmetros)
- **Qwen2:1.5b**: Modelo mais preciso (1.5B parâmetros) - pode ser lento em CPU
- **Qwen3:1.7b**: Modelo mais preciso (1.5B parâmetros) - pode ser lento em CPU

### ⚠️ Notas importantes:
- **IMPORTANTE**: Certifique-se de que o Docker esteja rodando: `docker compose up -d`
- Os modelos podem demorar 30-700s para responder
- Se der timeout, tente novamente ou use o TinyLLama
- Primeiro download dos modelos pode demorar vários minutos

In [ ]:
# 1. CONFIGURAÇÃO INICIAL
from chat_client import ChatClient
import time

# Configurar cliente
client = ChatClient(base_url="http://localhost:8000")

print("✅ Cliente configurado")
print("✅ URL da API:", client.base_url)

# Verificar status da API
try:
    health = client.health_check()
    print("✅ API funcionando:", health)
except:
    print("❌ API não está respondendo. Execute: docker compose up -d dentro de service")

# ESCOLHA OS MEDELOS PARA BAIXAR E FAZER TESTES

In [ ]:
modelos_a_baixar = ["tinyllama:latest", "tinydolphin:latest", "qwen2:1.5b","qwen3:1.7b"]

In [ ]:
# 2. VERIFICAR MODELOS DISPONÍVEIS
print("Verificando modelos disponíveis...")

# Listar modelos já instalados
modelos_disponiveis = client.listar_modelos()
print(f"Modelos instalados: {modelos_disponiveis}")

# Verificar se temos os modelos necessários
for modelo in modelos_a_baixar:
    if modelo in modelos_disponiveis:
        print(f"✓ {modelo}: disponível")
    else:
        print(f"✗ {modelo}: não encontrado, baixando")
        client.baixar_modelo(modelo)

print("\nPronto para usar os modelos disponíveis!")

# TESTE SEU PROMPT

In [ ]:
# aqui vai o texto com dados para extração
contexto = """
D.2 Trens Regionais
Plano Estratégico Ferroviário de Minas Gerais
Perfil da Proposta da Ferrovia para Transporte de Passageiros
Proposta: Araguari/ Campos Altos Código: RP 12-33
Categoria: Proposta Regional Versão: 00
Tipo de empreendimento:
Greenfield
Brownfield (via compartilhada com
operação da carga)
Características físicas:
Extensão (km): 503 Tipo bitola: Métrica
Total de estações: 8
Municípios atendidos (extensão acumulado em km):
Araguari 0,00
Uberlândia 46,7
Uberaba 175,34
Nova Ponte 247,11
Santa Juliana 269,67
Araxá 350,26
Ibiá 437,13
Campos Altos 503,00
Características operacionais:
Tempo de viagem ida (min): 551 Tempo de viagem ida & volta (min): 1.102
Viagens (mês): 27 Dias de operação (ano): 326
Demanda (ano): 470.930 Produção quilométrica (km/ano): 164.307
Tarifa do serviço:
Classe Econômica Classe Executiva
6,98 Tarifa fixa (R$) 18,72 Tarifa fixa (R$)
0,1447 Tarifa quilométrica (R$) 0,2685 Tarifa quilométrica (R$)
Desempenho da linha:
Receita anual (R$): 15.155.658
Pass,ano/km: 2,87 Receita,ano/km: 92,24
207 

 Plano Estratégico Ferroviário de Minas Gerais
Perfil da Proposta da Ferrovia para Transporte de Passageiros
Características da frota:
Total de composições: 1
Comprimento da composição (m): 163
Tipo carros Quantidade/composição Especificação
Locomotiva 1 Diesel
Carro de passageiros 4
Carros auxiliares 2
Capacidade (PAX/viagem): 285
Mapa de situação
208 PEF – Plano Estratégico Ferroviário de Minas Gerais | ANEXOS"   


""" 

In [ ]:
# 5. SUA CONSULTA PERSONALIZADA
# Modifique a pergunta abaixo e execute para suas proprias analises

# Escolha o modelo (tinyllama:latest é mais rapido e confiavel)
modelo = "tinyllama:latest"
modelo = "tinydolphin:latest"
# modelo = "qwen2:1.5b"
# modelo = "deepcoder:1.5b"
# modelo = "exaone-deep:2.4b"
# modelo = "qwen3:1.7b"

# Sua pergunta personalizada - MODIFIQUE AQUI:
# os dados sempres estarão assim:  <key>: <value>

print(f"Modelo selecionado: {modelo}")
print("\nProcessando...")

# Adicionar contexto para melhorar resposta
prompt      = """
você foi designado a responder o formulario <json> com base nos dados no <contexto>,
então responda com precisão, e para ajuda leia as <regras> abaixo:

# Regras:
<regras>
1. Responda com precisão e clareza.
2. Use os nomes exatos dos empreendimentos e estações conforme aparecem no texto.
3. Geralmente o dado esta logo depois dos dois pontos (:).
4. Atente-se a casos com mais de uma resposta possível, como em "Tipo de empreendimento" e "Estações atendidas".
5. A key do <json> está exatamente conforme o formulario, isso facilita a sua busca nos dados em <contexto>
6. Se não encontrar um dado, deixe o campo vazio ou com valor nulo.
7. Leia os comentarios ## com atenção, eles ajudam a entender o que deve ser preenchido.
8. Leia a mudança de topico com #, dentro do contexto também e dividido por tópicos.
9. Ao responder remova os comentarios ## e os divisores de tópicos #
</regras>

# Formulario JSON:
<json>
{{
    "Proposta": value1, ## NOME DA PROPOSTA QUE E DIFERENTE DO "Código"
    "Código": value2, ## CÓDIGO DA PROPOSTA QUE E DIFERENTE DA "Proposta"
    "Categoria": value3,
    "Tipo de empreendimento": ["empreendimento1", "empreendimento2", "empreendimento3" ...], ## RESPONDA COM OS NOMES DOS EMPREENDIMENTOS

    # Dados operacionais: 
    "Extensão (km)": value5,
    "Tipo bitola": value6,
    "Total de estações": value7,
    "Estações atendidas": ["estação1", "estação2", "estação3" ...], ## RESPONDA COM OS NOMES DE ESTAÇÕES

    # Demanda e receita:
    "Tempo de viagem ida (min)": value9,
    "Tempo de viagem ida & volta (min)": value10,
    "Viagens (mês)": value11,
    "Dias de operação (mês)": value12,
    "Demanda (mês)": value13,
    "Produção quilométrica (km/mês)": value14,
    "Tarifa do serviço": value15,

    # Desempenho da linha:
    "Receita anual (R$)": value16,
    "Pass.ano/km": value17,
    "Receita.ano/km": value18,
    
    # Características da frota:
    "Frota Total (operacional + reserva)": value19,
    "Total de carros de passageiros/trem": value20,
    "Tipo carros": ["carro_tipo1", "carro_tipo2", "carro_tipo3" ...], ## RESPONDA COM NOME DOS TIPOS DE CARROS
    "Quantidade/composição": [value21, value22, value23], ## RESPONDA COM OS NÚMERO PARA CADA TIPO CARRO
    "Especificação": ["especificação1", "especificação2", "especificação3" ...], ## RESPONDA COM AS ESPECIFICAÇÕES DOS TIPOS DE CARROS LISTADOS
    "Capacidade (PAX/viagem): value21
}}
</json>

# Fonte de dados:
<contexto> 
{}
</contexto> 

<tarefa>
extraia os dados da fonde de dados com a tag <contexto> e preencha o formulario com tag <json> nos campos value1, value2, etc.
</tarefa>

<resposta>
retorne o formulario <json> preenchido em formato json conforme em <json>, ao falar dele abra com <json> e ao terminar de falar dele feche com a tag </json>.
</resposta>
"""

try:
    inicio = time.time()
    resposta = client.chat(prompt, modelo=modelo, stream=False, timeout=6000)
    tempo = time.time() - inicio

    print(f"\nTempo: {tempo:.1f}s")
    print(f"\nResposta Especializada:")
    print("=" * 60)
    if 'resposta' in resposta:
        print(resposta['resposta'])
    elif 'erro' in resposta:
        print(f"Erro: {resposta['erro']}")
    else:
        print(str(resposta))
    print("=" * 60)
    
    print("\nPara fazer outra pergunta:")
    print("1. Modifique a variavel 'sua_pergunta' acima")
    print("2. Execute esta celula novamente")
except Exception as e:
    print(f"Erro: {str(e)}")
    print("Verifique se o Docker está rodando: docker compose up -d")

# EXTRAINDO DADOS DO PDF E PREENCHENDO NO JSON COM LLM

In [ ]:
from pdf_processor import PDFReader
import json

# Adicionar contexto para melhorar resposta
prompt      = """
você foi designado a responder o formulario <json> com base nos dados no <contexto>,
então responda com precisão, e para ajuda leia as <regras> abaixo:

# Regras:
<regras>
1. Responda com precisão e clareza.
2. Use os nomes exatos dos empreendimentos e estações conforme aparecem no texto.
3. Geralmente o dado esta logo depois dos dois pontos (:).
4. Atente-se a casos com mais de uma resposta possível, como em "Tipo de empreendimento" e "Estações atendidas".
5. A key do <json> está exatamente conforme o formulario, isso facilita a sua busca nos dados em <contexto>
6. Se não encontrar um dado, deixe o campo vazio ou com valor nulo.
7. Leia os comentarios ## com atenção, eles ajudam a entender o que deve ser preenchido.
8. Leia a mudança de topico com #, dentro do contexto também e dividido por tópicos.
9. Ao responder remova os comentarios ## e os divisores de tópicos #
</regras>

# Formulario JSON:
<json>
{{
    "Proposta": value1, ## NOME DA PROPOSTA QUE E DIFERENTE DO "Código"
    "Código": value2, ## CÓDIGO DA PROPOSTA QUE E DIFERENTE DA "Proposta"
    "Categoria": value3,
    "Tipo de empreendimento": ["empreendimento1", "empreendimento2", "empreendimento3" ...], ## RESPONDA COM OS NOMES DOS EMPREENDIMENTOS

    # Dados operacionais: 
    "Extensão (km)": value5,
    "Tipo bitola": value6,
    "Total de estações": value7,
    "Estações atendidas": ["estação1", "estação2", "estação3" ...], ## RESPONDA COM OS NOMES DE ESTAÇÕES

    # Demanda e receita:
    "Tempo de viagem ida (min)": value9,
    "Tempo de viagem ida & volta (min)": value10,
    "Viagens (mês)": value11,
    "Dias de operação (mês)": value12,
    "Demanda (mês)": value13,
    "Produção quilométrica (km/mês)": value14,
    "Tarifa do serviço": value15,

    # Desempenho da linha:
    "Receita anual (R$)": value16,
    "Pass.ano/km": value17,
    "Receita.ano/km": value18,
    
    # Características da frota:
    "Frota Total (operacional + reserva)": value19,
    "Total de carros de passageiros/trem": value20,
    "Tipo carros": ["carro_tipo1", "carro_tipo2", "carro_tipo3" ...], ## RESPONDA COM NOME DOS TIPOS DE CARROS
    "Quantidade/composição": [value21, value22, value23], ## RESPONDA COM OS NÚMERO PARA CADA TIPO CARRO
    "Especificação": ["especificação1", "especificação2", "especificação3" ...], ## RESPONDA COM AS ESPECIFICAÇÕES DOS TIPOS DE CARROS LISTADOS
    "Capacidade (PAX/viagem): value21
}}
</json>

# Fonte de dados:
<contexto> 
{}
</contexto> 

<tarefa>
extraia os dados da fonde de dados com a tag <contexto> e preencha o formulario com tag <json> nos campos value1, value2, etc.
</tarefa>

<resposta>
retorne o formulario <json> preenchido, ao falar dele abra com <json> e ao terminar de falar dele feche com a tag </json>.
</resposta>
"""

modelo      = "qwen3:1.7b"

path_pdf    = "data\\external\\Relatorio_PEF_Minas_2021_ANEXOS.pdf"
PDFTool     = PDFReader(path_pdf, verbose=False)

results     = {}
for page in range(207+2, 244+2, 2):
    if page not in results:
        results[page]   = {}
    text_start_page     = PDFTool.extract_text_from_page(page)
    text_next_page      = PDFTool.extract_text_from_page(page+1)
    if text_start_page and text_next_page:
        text_start_page = text_start_page[0]
        text_next_page  = text_next_page[0]
        if "text" in text_start_page and "text" in text_next_page:
            results[page]["text"] = text_start_page["text"] + " \n\n " + text_next_page["text"]
            resposta    = client.chat(prompt.format(results[page]["text"]), modelo=modelo, stream=False, timeout=6000)
            results[page]["parsed_llm"] = resposta["resposta"]
            with open("data\\interim\\PARSED_LLM.json", "w", encoding="utf-8") as file:
                json.dump(results, file, ensure_ascii=False)

# TRANSFORMANDO O JSON EM EXCEL

In [ ]:
import pandas as pd
import json
lines       = []
pages_error = {}
with open("data\\interim\\PARSED_LLM.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    for page in data.keys():
        if "<json>" in data[page]["parsed_llm"]:
            tags = ("<json>", "</json>")
        else:# "```json" in data[page]["parsed_llm"]:
            tags = ("```json", "```")
        try:
            text_parsed = "{"+ data[page]["parsed_llm"].split(tags[0])[1].split(tags[1])[0].strip().split("{")[1].split("}")[0] + "}"
            lines.append(json.loads(text_parsed))
        except (IndexError, json.JSONDecodeError):
            pages_error[page] = {"parsed_llm": data[page]["parsed_llm"]}
            print(f"Error parsing page {page}")
pd.DataFrame(lines).to_csv("Extract_Data_PEF.csv", sep=";", index=False, encoding="ansi")